In [1]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.7 MB/s eta 0:00:00


1. Загружаем набор данных о диабете и разделяем его на обучающую и тестовую выборки.
2. Определяем функцию `objective`, которая принимает `trial` и настраивает гиперпараметр `alpha` для Ridge-регрессии. Мы используем `suggest_loguniform`, чтобы задать логарифмическое пространство для `alpha`.
3. Обучаем модель и вычисляем MSE, который будем минимизировать.
4. Создаем объект `study`, чтобы оптимизировать гиперпараметры, и запускаем процесс оптимизации с заданным количеством операций (в данном случае 100).
5. После нахождения лучших гиперпараметров, мы обучаем финальную модель и рассчитываем R² и MSE на тестовых данных.
6. В конце выводим лучшие гиперпараметры, а также значения R² и MSE.

In [2]:
import numpy as np
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Загружаем данные
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

In [4]:
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Функция для оптимизации гиперпараметров
def objective(trial):
    # Определяем гиперпараметры
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
    model = Ridge(alpha=alpha)

    # Обучаем модель
    model.fit(X_train, y_train)

    # Предсказываем
    y_pred = model.predict(X_test)

    # Вычисляем ошибки
    mse = mean_squared_error(y_test, y_pred)
    return mse  # Возвращаем MSE для минимизации

In [6]:
# Настраиваем оптимизацию гиперпараметров
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Выводим лучшие гиперпараметры
best_params = study.best_params
best_model = Ridge(alpha=best_params['alpha'])
best_model.fit(X_train, y_train)

# Делаем предсказания на тестовом наборе
y_pred = best_model.predict(X_test)

[I 2025-02-23 17:08:47,693] A new study created in memory with name: no-name-d04ed672-7377-4437-8ae2-a5f6be5a3aa1
<ipython-input-5-d8cd7d04daba>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
[I 2025-02-23 17:08:47,736] Trial 0 finished with value: 2865.4508605802157 and parameters: {'alpha': 0.05313571264070418}. Best is trial 0 with value: 2865.4508605802157.
<ipython-input-5-d8cd7d04daba>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)
[I 2025-02-23 17:08:47,743] Trial 1 finished with value: 2900.1125718608523 and parameters: {'alpha': 1.5505328686392524e-05}.

In [7]:
# Вычисляем R² и MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Лучшие гиперпараметры: alpha = {best_params['alpha']}")
print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")

Лучшие гиперпараметры: alpha = 0.15491959314495785
R²: 0.4615
MSE: 2853.2739
